In [14]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import holidays as pyholidays

# read data from data folder (organized)

In [15]:
df=pd.read_excel('/Users/user/Documents/HHCRSP/proyecto_HHCRSP/data/Instance_organized.xlsx',sheet_name='5')
# convert string to list
df["DLI"]=df["DLI"].apply(lambda x: x.split(","))
df["DLI"]=df["DLI"].apply(lambda x: [int(i) for i in x])
# find position of 1's in DLI
df["DLI"]=df["DLI"].apply(lambda x: np.array([i+1 for i, e in enumerate(x) if e == 1]))

#df["vacations"].apply(lambda x: x.split(",") if type(x) == str else [])
df["vacations"]=df["vacations"].apply(lambda x: np.array([int(i) for i in x.split(",")]) if type(x) == str else [])#
display(df)

,ID,name,vacations,TF,morning_availability_labor_day,morning_availability_weekend,afternoon_availability_labor_day,afternoon_availability_weekend,DLI,HA
0,1,Otalora_Castro_Rocio_del_Pilar,[],0,1,1,0,1,"[8, 14, 21, 22, 28]",-53.0
1,2,Lina_Mabel_Torres_Pulido,"[1, 2]",0,1,1,0,1,"[8, 15]",-21.0
2,3,Rodriguez_Bohorquez_Luz_Dary,[],1,1,1,0,1,"[8, 14, 21, 22, 28]",-33.5
3,4,Carrero_Contreras_Tania_Maryely,"[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2...",1,0,1,1,1,"[1, 7, 8, 21, 29]",-51.5
4,5,Ballesteros_Mesa_Jeimy_Catalina,[],1,0,1,1,1,"[1, 7, 14, 15, 29]",-60.5
5,6,Barrera_Castillo_Jenny_Lorena,[],0,1,1,1,1,"[8, 14, 22, 28]",-48.5
6,7,Murcia_Maira_Alejandra,[],0,0,1,0,1,"[1, 7, 15, 21, 28, 29]",0.0
7,8,Molano_Tovar_Sandra_Johana,[],1,0,1,1,1,"[1, 7, 15, 21, 28, 29, 31]",-18.0


In [16]:

df.rename(columns={"ID":"nurse_id",
                   "name":"nurse_name",
                   "vacations":"vacations",
                   "DLI":"dates_off",
                   "TF":"shift_preference",
                   "HA":"accumulated_hours",
                   "vacations":"vacations"
                   },inplace=True)

#df["dates_off"]=df["dates_off"].apply(lambda x: x.split(",") if type(x) == str else [])
#df["dates_off"]=df["dates_off"].apply(lambda x: [int(i) for i in x] if type(x) == list else [])


dict_preference={0:"morning",1:"afternoon"}
df["shift_preference"]=df["shift_preference"].apply(lambda x: dict_preference[x])
df.reset_index(inplace=True,drop=True)
df.columns

Index(['nurse_id', 'nurse_name', 'vacations', 'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours'],
      dtype='object')

In [17]:
df[['nurse_id', 'nurse_name',  'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours','vacations']].to_json('nurses.json',orient='records')

In [18]:
shift_type={"morning":["M", "COM", "DISP"],"afternoon":["T1", "T2", "CHX"]}

# demands dictionary has a first key representing the shift type, a second key representing the day of the week, and a third  separating
# holidays and not holidays using 1 to indicate holiday and 0 to indicate not holiday

def random_demands_generator(shift_type):

    demands={}
    for shift in shift_type.keys():
        morning_max_demand=np.random.randint(2,5)
        afternoon_max_demand=np.random.randint(1,morning_max_demand)
        for st in shift_type[shift]:
            for day in range(7):
                for holiday in [0,1]:
                    if shift=="morning":
                        if st=="M":
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=morning_max_demand
                            else:
                                demands[(st,day,holiday)]=morning_max_demand-1
                        elif st=="COM":
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=morning_max_demand-1
                            else:
                                demands[(st,day,holiday)]=morning_max_demand-2
                        else:
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=0
                            else:
                                demands[(st,day,holiday)]=max(morning_max_demand-2,1)
                    if shift=="afternoon":
                        if st=="T1":
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=afternoon_max_demand
                            else:
                                demands[(st,day,holiday)]=afternoon_max_demand
                        elif st=="T2":
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=max(afternoon_max_demand-1,1)
                            else:
                                demands[(st,day,holiday)]=max(afternoon_max_demand-1,0)
                        else:
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=max(morning_max_demand-1,1)  
                            else:
                                demands[(st,day,holiday)]=0                  

    return demands

demands={
    ("M",0,0):2,
    ("M",0,1):1,
    ("M",1,0):2,
    ("M",1,1):1,
    ("M",2,0):2,
    ("M",2,1):1,
    ("M",3,0):2,
    ("M",3,1):1,
    ("M",4,0):2,
    ("M",4,1):1,
    ("M",5,0):1,
    ("M",5,1):1,
    ("M",6,0):1,
    ("M",6,1):1,
    ("COM",0,0):1,
    ("COM",0,1):1,
    ("COM",1,0):1,
    ("COM",1,1):1,
    ("COM",2,0):1,
    ("COM",2,1):1,
    ("COM",3,0):1,
    ("COM",3,1):1,
    ("COM",4,0):1,
    ("COM",4,1):1,
    ("COM",5,0):0,
    ("COM",5,1):0,
    ("COM",6,0):0,
    ("COM",6,1):0,
    ("DISP",0,0):0,
    ("DISP",0,1):1,
    ("DISP",1,0):0,
    ("DISP",1,1):1,
    ("DISP",2,0):0,
    ("DISP",2,1):1,
    ("DISP",3,0):0,
    ("DISP",3,1):1,
    ("DISP",4,0):0,
    ("DISP",4,1):1,
    ("DISP",5,0):1,
    ("DISP",5,1):1,
    ("DISP",6,0):1,
    ("DISP",6,1):1,
    ("T1",0,0):1,
    ("T1",0,1):1,
    ("T1",1,0):1,
    ("T1",1,1):1,
    ("T1",2,0):1,
    ("T1",2,1):1,
    ("T1",3,0):1,
    ("T1",3,1):1,
    ("T1",4,0):1,
    ("T1",4,1):1,
    ("T1",5,0):1,
    ("T1",5,1):1,
    ("T1",6,0):1,
    ("T1",6,1):1,
    ("T2",0,0):1,
    ("T2",0,1):0,
    ("T2",1,0):1,
    ("T2",1,1):0,
    ("T2",2,0):1,
    ("T2",2,1):0,
    ("T2",3,0):1,
    ("T2",3,1):0,
    ("T2",4,0):1,
    ("T2",4,1):0,
    ("T2",5,0):0,
    ("T2",5,1):0,
    ("T2",6,0):0,
    ("T2",6,1):0,
    ("CHX",0,0):1,
    ("CHX",0,1):0,
    ("CHX",1,0):1,
    ("CHX",1,1):0,
    ("CHX",2,0):1,
    ("CHX",2,1):0,
    ("CHX",3,0):1,
    ("CHX",3,1):0,
    ("CHX",4,0):1,
    ("CHX",4,1):0,
    ("CHX",5,0):0,
    ("CHX",5,1):0,
    ("CHX",6,0):0,
    ("CHX",6,1):0
}



In [19]:
list_shifts=[]
# list of dates of january 2023
dates = pd.date_range(start='5/1/2023', end='5/31/2023')
shift_id=0
for fecha in dates:
    for shift in shift_type:
        for jornada in shift_type[shift]:
            list_shifts.append([shift_id, 
                                fecha.strftime("%Y-%m-%d"), 
                                fecha.week,
                                jornada, shift,fecha.weekday(),
                                fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023),
                                demands[(jornada,fecha.weekday(),int( fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023)))]
                                ])
            shift_id+=1
df_shifts=pd.DataFrame(list_shifts,columns=["shift_id","shift_date","week_of_year","shift","shift_type","weekday","holiday","demand"])
df_shifts.index=df_shifts["shift_id"]



In [20]:
df_shifts.to_json('shifts.json',orient='records',index=True)

# Fake instances generator

In [25]:
pd.set_option('display.max_rows', 500)
df_instance=pd.read_excel('/Users/user/Documents/HHCRSP/proyecto_HHCRSP/data/instances_for_resampling.xlsx', sheet_name='8')

# convert string to list
df_instance["DLI"]=df_instance["DLI"].apply(lambda x: x.split(","))
df_instance["DLI"]=df_instance["DLI"].apply(lambda x: [int(i) for i in x])
# find position of 1's in DLI
df_instance["DLI"]=df_instance["DLI"].apply(lambda x: np.array([i+1 for i, e in enumerate(x) if e == 1]))

#df_instance["vacations"].apply(lambda x: x.split(",") if type(x) == str else [])
df_instance["vacations"]=df_instance["vacations"].apply(lambda x: np.array([int(i) for i in x.split(",")]) if type(x) == str else [])#


# select a random subset of rows
df_instance=df_instance.sample(n=14, random_state=1)
df_instance['name']=["person_"+str(i) for i in range(1,df_instance.shape[0]+1)]

df_instance.rename(columns={"ID":"nurse_id",
                   "name":"nurse_name",
                   "vacations":"vacations",
                   "DLI":"dates_off",
                   "TF":"shift_preference",
                   "HA":"accumulated_hours",
                   "vacations":"vacations"
                   },inplace=True)

#df_instance["dates_off"]=df_instance["dates_off"].apply(lambda x: x.split(",") if type(x) == str else [])
#df_instance["dates_off"]=df_instance["dates_off"].apply(lambda x: [int(i) for i in x] if type(x) == list else [])


dict_preference={0:"morning",1:"afternoon"}
df_instance["shift_preference"]=df_instance["shift_preference"].apply(lambda x: dict_preference[x])
df_instance.reset_index(inplace=True,drop=True)
df_instance[['nurse_id', 'nurse_name',  'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours','vacations']].to_json('nurses.json',orient='records')

In [22]:
df_instance[['nurse_id', 'nurse_name',  'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours','vacations']].head()

,nurse_id,nurse_name,shift_preference,morning_availability_labor_day,morning_availability_weekend,afternoon_availability_labor_day,afternoon_availability_weekend,dates_off,accumulated_hours,vacations
0,45,person_1,afternoon,0,1,1,1,"[1, 7, 14, 15, 29]",-60.5,[]
1,3,person_2,afternoon,1,1,0,1,"[5, 13, 19, 20, 27]",-7.0,[]
2,47,person_3,morning,0,1,0,1,"[1, 7, 15, 21, 28, 29]",0.0,[]
3,20,person_4,morning,0,1,1,1,[],-19.0,[]
4,33,person_5,morning,1,1,0,1,"[8, 14, 21, 22, 28]",-53.0,[]


In [23]:
import numpy as np
import calendar
import datetime
def create_random_instance(n_nurses:int,seed:int=None):
    """
    Create a random instance of the nurse scheduling problem.

    return: pandas dataframe with columns 'nurse_id', 'nurse_name',  'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours','vacations'
    """
    if seed != None:
        np.random.seed(seed)
    np.random.seed(seed)
    nurse_id = np.arange(n_nurses)
    nurse_name = np.array([f'Nurse {i}' for i in nurse_id])
    shift_preference = ['afternoon' if np.random.rand() < 0.5 else 'morning' for i in nurse_id]
    morning_availability_labor_day = np.random.randint(0,2,n_nurses)
    morning_availability_weekend = np.random.randint(0,2,n_nurses)
    afternoon_availability_labor_day = np.random.randint(0,2,n_nurses)
    afternoon_availability_weekend = np.random.randint(0,2,n_nurses)

    month=np.random.randint(1,13)
    year=2022

    #enumerate weekend days of the month
    weekend_days = []
    for i in range(1,calendar.monthrange(year,month)[1]+1):
        if datetime.date(year,month,i).weekday() > 4:
            weekend_days.append(i)
    
    # select random weekend days
    dates_off = []
    for i in range(n_nurses):
        dates_off.append(np.random.choice(weekend_days, size=np.random.randint(0,len(weekend_days)), replace=False))
    accumulated_hours = list(np.random.randint(-30,30,n_nurses))
    vacations = []
    for i in range(n_nurses):
        if np.random.rand() < 0.2:
            vacations_duration = np.random.randint(7,15)
            start_date = np.random.randint(1,calendar.monthrange(year,month)[1]+1)
            vacations.append([start_date+j for j in range(vacations_duration) if start_date+j<32])
        else:
            vacations.append([])
    return pd.DataFrame({'nurse_id':nurse_id,
                            'nurse_name':nurse_name,
                            'shift_preference':shift_preference,
                            'morning_availability_labor_day':morning_availability_labor_day,
                            'morning_availability_weekend':morning_availability_weekend,
                            'afternoon_availability_labor_day':afternoon_availability_labor_day,
                            'afternoon_availability_weekend':afternoon_availability_weekend,
                            'dates_off':dates_off,
                            'accumulated_hours':accumulated_hours,
                            'vacations':vacations})

df_random=create_random_instance(16)
df_random.to_json('nurses.json',orient='records')
                               


# Feasibility checker

In [24]:

from HHCRS_multiobjective_model import check_feasibility

print(check_feasibility("../data/"))

ModuleNotFoundError: No module named 'app'